In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Heart_rate"
cohort = "GSE117070"

# Input paths
in_trait_dir = "../../input/GEO/Heart_rate"
in_cohort_dir = "../../input/GEO/Heart_rate/GSE117070"

# Output paths
out_data_file = "../../output/preprocess/Heart_rate/GSE117070.csv"
out_gene_data_file = "../../output/preprocess/Heart_rate/gene_data/GSE117070.csv"
out_clinical_data_file = "../../output/preprocess/Heart_rate/clinical_data/GSE117070.csv"
json_path = "../../output/preprocess/Heart_rate/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"The Heritage family study - skeletal muscle gene expression"
!Series_summary	"Gene expression profiles generated from skeletal muscle biopsies taken from participants of the HERITAGE family study. Participants completed an endurance training regime in which a skeletal muscle biopsy was taken prior to the start and after the final session of the program. Biopsies were used to generate Affymetrix gene expression microarrays."
!Series_overall_design	"The experimental design and exercise training protocol of the HERITAGE Family Study have been described previously (Bouchard et al., 1995). Participants were sedentary at baseline and normotensive. Each participant exercised three times per week for 20 weeks on cycle ergometers controlled by direct heart rate (HR) monitoring. Muscle biopsies of vastus lateralis were obtained at baseline and post-training."
Sample Characteristics Dictionary:
{0: ['status: pre-training', 'status: post-training']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any

# 1. Assess gene expression data availability
# Based on background information, this is a gene expression study using Affymetrix microarrays
is_gene_available = True

# 2. Variable availability and data type conversion
# 2.1 Data Availability

# For trait (Heart_rate): 
# The dataset doesn't explicitly mention heart rate measurements in sample characteristics
# However, from background information, this is a training study where heart rate monitoring
# was used, but the variable itself isn't provided as a direct measurement in the data
trait_row = None

# For age: No age information appears to be available in the sample characteristics
age_row = None

# For gender: No gender information appears to be available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
# Define conversion functions for completeness, even though we don't have these variables

def convert_trait(value):
    # Not applicable as trait data is not available
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        return float(value)  # Heart rate would typically be continuous
    except:
        return None

def convert_age(value):
    # Not applicable as age data is not available
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    # Not applicable as gender data is not available
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if value in ['female', 'f', 'woman']:
        return 0
    elif value in ['male', 'm', 'man']:
        return 1
    else:
        return None

# 3. Save metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step since trait_row is None


A new JSON file was created at: ../../output/preprocess/Heart_rate/cohort_info.json


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 60
Header line: "ID_REF"	"GSM3270055"	"GSM3270056"	"GSM3270057"	"GSM3270058"	"GSM3270059"	"GSM3270060"	"GSM3270061"	"GSM3270062"	"GSM3270063"	"GSM3270064"	"GSM3270065"	"GSM3270066"	"GSM3270067"	"GSM3270068"	"GSM3270069"	"GSM3270070"	"GSM3270071"	"GSM3270072"	"GSM3270073"	"GSM3270074"	"GSM3270075"	"GSM3270076"	"GSM3270077"	"GSM3270078"	"GSM3270079"	"GSM3270080"	"GSM3270081"	"GSM3270082"	"GSM3270083"	"GSM3270084"	"GSM3270085"	"GSM3270086"	"GSM3270087"	"GSM3270088"	"GSM3270089"	"GSM3270090"	"GSM3270091"	"GSM3270092"	"GSM3270093"	"GSM3270094"	"GSM3270095"	"GSM3270096"	"GSM3270097"	"GSM3270098"	"GSM3270099"	"GSM3270100"	"GSM3270101"	"GSM3270102"	"GSM3270103"	"GSM3270104"	"GSM3270105"	"GSM3270106"	"GSM3270107"	"GSM3270108"	"GSM3270109"	"GSM3270110"	"GSM3270111"	"GSM3270112"	"GSM3270113"	"GSM3270114"	"GSM3270115"	"GSM3270116"	"GSM3270117"	"GSM3270118"	"GSM3270119"	"GSM3270120"	"GSM3270121"	"GSM3270122"	"GSM3270123"	"GSM3270124"	"GSM3270125"	"GSM3270126"	"GSM3270127"	

Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers in the header
# Examples: "1007_s_at", "1053_at", "117_at", etc.
# These appear to be Affymetrix probe IDs (likely HG-U133 array)
# They are not standard human gene symbols and will need to be mapped

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE117070
Line 6: !Series_title = The Heritage family study - skeletal muscle gene expression
Line 7: !Series_geo_accession = GSE117070
Line 8: !Series_status = Public on Jul 14 2018
Line 9: !Series_submission_date = Jul 13 2018
Line 10: !Series_last_update_date = Dec 15 2021
Line 11: !Series_pubmed_id = 29117557
Line 12: !Series_pubmed_id = 34857871
Line 13: !Series_summary = Gene expression profiles generated from skeletal muscle biopsies taken from participants of the HERITAGE family study. Participants completed an endurance training regime in which a skeletal muscle biopsy was taken prior to the start and after the final session of the program. Biopsies were used to generate Affymetrix gene expression 


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DE

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe identifiers in gene expression data and annotation data
# The gene expression data uses probe IDs like "1007_s_at" as identifiers
# The annotation data has column "ID" that matches these probe IDs
# The "Gene Symbol" column contains the human gene symbols we need

# 2. Get a gene mapping dataframe
# We need to extract the "ID" and "Gene Symbol" columns
prob_col = "ID"
gene_col = "Gene Symbol"

# Create the mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply gene mapping to convert probe-level measurements to gene expression
# We'll use the library function that handles the many-to-many mapping properly
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the shape of the resulting gene expression data
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print(f"First 10 gene symbols: {gene_data.index[:10].tolist()}")


Gene expression data shape after mapping: (21278, 82)
First 10 gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data_normalized.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Let's check if the clinical data from the first steps has usable heart rate information
# From Step 1, we saw clinical data has only one row with "status: pre-training", "status: post-training"
print("\nSample characteristics:")
print(sample_characteristics_dict)

# Since there's no explicit heart rate data available, we'll indicate that trait is not available
is_gene_available = True
is_trait_available = False  # Heart rate data is not directly available in this dataset

# 3. Save metadata about the dataset
# Since this is initial filtering (trait data not available), we should use is_final=False
note = "Dataset contains gene expression data from skeletal muscle samples before and after endurance training. Heart rate was used for monitoring during training but not available as a direct measurement in the dataset."

is_usable = validate_and_save_cohort_info(
    is_final=False,  # Using False since we're doing initial filtering based on trait availability
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

print(f"Dataset validated and metadata saved. Usable: {is_usable}")
print("No linked data to save since trait data is unavailable")

Gene data shape after normalization: (19845, 82)


Normalized gene data saved to ../../output/preprocess/Heart_rate/gene_data/GSE117070.csv

Sample characteristics:
{0: ['status: pre-training', 'status: post-training']}
Dataset validated and metadata saved. Usable: False
No linked data to save since trait data is unavailable
